 
Authors: Federico Gritti, Kadir Oezden, Funni Veh

# Project Summary: Sorting Robot with a Twist
## Objective
Develop an algorithm that enables a robot arm to identify and sort objects (e.g., colored cubes) into designated bins based on their color.
## Robot Platform
Utilize the Panda robot manipulator, assuming knowledge of the objects colors.
## Algorithm Steps
For each detected object $i$:
- Compute the picking position $p_{pick}$ and the placing position $p_{place}$ based on the object's color.
- Use inverse kinematics to obtain the robot configurations $q_{pick}$ and $q_{place}$.
## Control Strategies
- **Joint-Space Trajectory Interpolation**: Move from pick to place positions using interpolated joint trajectories between $q_{pick}$ and $q_{place}$.
- **Resolved-Rate Motion Control**: Use Cartesian velocity control to reach pick and place poses smoothly.
## Analysis
- Plot and compare trajectories for both strategies (joint and Cartesian space).
- Analyze end-effector path smoothness, execution time, and sorting accuracy.
## Extra Steps: Camera Integration
Simulate a camera to enable the robot to find the location and color of the spheres based on visual input. The sensing can be done in two ways:
1. **OpenCV Processing**:
   - Use OpenCV to process the camera image.
   - Apply HSV color masking to find the pixel coordinates of each berry.
2. **Convolutional Neural Network (CNN)**:
   - Use a CNN that takes the raw camera image as direct input.
   - Train the CNN to learn its own policy for "seeing" the berries.  


# Introduction
In this notebook, we present our project on developing a sorting robot for berries. The robot is designed to identify and sort different types of berries based on their color, size, and quality using computer vision and machine learning techniques.

In [ ]:

import swift
import roboticstoolbox as rtb
import spatialgeometry as sg
import spatialmath as sm
import numpy as np
import time

blue = [0, 0, 1]
red = [1, 0, 0]
green = [0, 1, 0]
orange = [1, 0.5, 0]
colors = [blue, red, green]

light_red = [0.9, 0.7, 0.7]
light_green = [0.7, 0.9, 0.7]
light_blue = [0.7, 0.7, 0.9]


class Berry:
    def __init__(self, radius, table_position_x, table_position_y, table_position_z, table_height):
        self.table_position_z = table_position_z
        self.table_height = table_height
        self.radius = radius
        self.random_color = colors[np.random.choice(len(colors))]
        random_x_position_on_table = np.random.uniform(table_position_x - table_depth/2 + self.radius, table_position_x + table_depth/2 - self.radius)
        random_y_position_on_table = np.random.uniform(table_position_y - table_width/2 + self.radius, table_position_y + table_width/2 - self.radius)
        self.std_pose_z = table_position_z + table_height/2 + self.radius
        random_position = sm.SE3(random_x_position_on_table, random_y_position_on_table, self.std_pose_z)
        self._sphere = sg.Sphere(radius=radius, pose=random_position, color=self.random_color)

    @property
    def pose(self):
        return sm.SE3(self._sphere.T)  # Return an SE3 object from the sphere's transformation matrix

    @property
    def swift(self):
        return self._sphere
    
    def eat(self):
        self._sphere.T = sm.SE3(0, 0, 1000).A  # Use .A to get the 4x4 numpy array
    
    def set_position(self, position):
        self._sphere.T = position.A  # Use .A to get the 4x4 numpy array from SE3

class Box:
    def __init__(self, width, depth, height, position, color=[0.9, 0.9, 0.9], wall_thickness=0.02):
        self.width = width
        self.depth = depth
        self.height = height
        self.position = position
        self.color = color
        self.wall_thickness = wall_thickness
        self.boxes = []
        bottom = sg.Cuboid(scale=[width, depth, wall_thickness],pose=sm.SE3(position.x, position.y, position.z - height/2 + wall_thickness/2), color=color)
        self.boxes.append(bottom)
        front = sg.Cuboid(scale=[width, wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x, position.y - depth/2 + wall_thickness/2, position.z),color=color)
        self.boxes.append(front)
        back = sg.Cuboid(scale=[width, wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x, position.y + depth/2 - wall_thickness/2, position.z), color=color)
        self.boxes.append(back)
        left = sg.Cuboid(scale=[wall_thickness, depth - 2*wall_thickness, height - 2*wall_thickness],pose=sm.SE3(position.x - width/2 + wall_thickness/2, position.y, position.z),color=color)
        self.boxes.append(left)
        right = sg.Cuboid(scale=[wall_thickness, depth - 2*wall_thickness, height - 2*wall_thickness], pose=sm.SE3(position.x + width/2 - wall_thickness/2, position.y, position.z), color=color)
        self.boxes.append(right)

    @property
    def swift(self):
        return self.boxes
    
class Table:    
    def __init__(self, x, y, z, depth, width, height, color):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
        self.depth = float(depth)
        self.width = float(width)
        self.height = float(height) 
        self._cuboid = sg.Cuboid(scale=[self.depth, self.width, self.height], pose=sm.SE3(self.x, self.y, self.z), color=color)
    
    @property
    def table_position_z(self):
        return float(self.z)
    
    @property
    def table_height(self):
        return float(self.height)
    
    @property
    def swift(self):
        return self._cuboid
    
    @swift.setter
    def swift(self, swift):
        self._cuboid = swift

class Robot:
    def __init__(self):
        self.dt=0.01 # time step
        self.error = np.ones((3,1)) # initial error
        self.gain = 1.5 # control gain
        self.dumping_factor = 0.15

        self._panda = rtb.models.Panda()
        self.rest_pose = np.array([-9.73143718e-04, -3.71976205e-01, -8.31161349e-03, -2.10634318e+00, 3.37384544e-02,  2.02548309e+00,  7.85398160e-01])
        self._panda.q = self.rest_pose

    @property
    def swift(self):
        return self._panda


    def move_to_target(self, target):
        self.error = target.t - self._panda.fkine(self._panda.q).t # the position error only
        J =  self._panda.jacob0(self._panda.q)[0:3,:] # the position part of the Jacobian
        J_pinv = np.linalg.pinv(J) # this is the standard pseudoinverse which means no damping
        J_damped_pinv = J.T @ np.linalg.inv(J@J.T + self.dumping_factor * np.eye(3))
        J_pinv_ = J_damped_pinv # toggling between damped and standard pseudoinverse
        cond_number = np.linalg.cond(J_pinv_)
        q_dot = self.gain * J_pinv_ @ self.error
        self._panda.qd = q_dot
    
    def set_robot_position(self, position):
        self._panda.q = position

    def reset_error(self):
        self.error = np.ones((3,1))

class Environment:
    def __init__(self):
        self.swift_env = swift.Swift()
        self.swift_env.launch(realtime=True,comms="rtc",browser="notebook")
        self.berries = []

    def add_multiple_objects_swift(self, objects):
        for obj in objects:
            self.swift_env.add(obj)



if __name__ == "__main__":

    env = Environment()
    
    table_height = 0.05 # height of the tablet
    table_width = 1
    table_depth = 0.6
    table_position_x = 0.5
    table_position_y = 0.0
    table_position_z = 0.25
    
    table2_position_x = -0.5
    table2_position_y = 0.0
    table2_position_z = 0.25
    
    box1_position_x = -0.5
    box1_position_y = -0.25
    wall_thickness = 0.02

    box1_position_z = table_position_z + table_height + wall_thickness

    box_width = 0.2
    box_depth = 0.2
    box_height = 0.1
    table1 = Table(table_position_x, table_position_y, table_position_z, table_depth, table_width, table_height, orange)
    table2 = Table(table2_position_x, table2_position_y, table2_position_z, table_depth, table_width, table_height, orange)
    box1 = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x, box1_position_y , box1_position_z), color=light_red, wall_thickness=wall_thickness)
    box2 = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x, box1_position_y + box_width + 0.05, box1_position_z), color=light_green, wall_thickness=wall_thickness)
    box3 = Box(box_width, box_depth, box_height, sm.SE3(box1_position_x, box1_position_y + 2*(box_width + 0.05), box1_position_z), color=light_blue, wall_thickness=wall_thickness)


    robot = Robot()


    env.swift_env.add(robot.swift)
    env.swift_env.add(table1.swift)
    env.swift_env.add(table2.swift)
    env.add_multiple_objects_swift(box1.boxes)
    env.add_multiple_objects_swift(box2.boxes)
    env.add_multiple_objects_swift(box3.boxes)
    env.swift_env.step(1)

    # berry = Berry(0.02, table_position_x, table_position_y, table_position_z, table_height)
    # env.swift_env.add(berry.swift)
    # env.swift_env.step(0.01)
    # berry.eat()
    # env.swift_env.step(0.01)





total_berries = 100

for i in range(total_berries):
    berry_radius = 0.02
    berry = Berry(berry_radius, table_position_x, table_position_y, table_position_z, table_height)
    env.swift_env.add(berry.swift)
    env.berries.append(berry)

for berry in env.berries:
    time.sleep(1)
    while np.linalg.norm(robot.error) > 0.02 :
        robot.move_to_target(berry.pose)
        env.swift_env.step(robot.dt)
        if np.linalg.norm(robot.error) <= 0.02:
            total_berries -= 1
            berry.eat()
            print(f"Berries remaining: {total_berries}")
            robot.reset_error()
            break

Berries remaining: 4
Berries remaining: 3
Berries remaining: 2
Berries remaining: 1
Berries remaining: 0
